# Tutorial

This tutorial demonstrates how to sample EOS parameters from BNS data. The input is a set of MCMC runs for each BNS event that contain the parameters ($\mathcal{M}$, $q$, $\tilde\Lambda$). The code makes the assumption that the MCMC runs were done using flat (uniform) priors in $q$ and $\tilde\Lambda$. If the priors were not flat to begin with, you must reweight the posterior in such a way that the reweighted samples correspond to a flat prior.

## Dependencies

To run eosinference, you will have to install the following packages:
 * pandas: `pip install --user pandas`
 * h5py: `pip install --user h5py`
 * emcee (I used version 2.2.1): `pip install --user emcee`
 * corner: `pip install --user corner`
 * lalsuite: I'm sorry you have to install this.


## Estimating EOS parameters and NS properties from multiple BNS events

You will have to run 4 scripts to produce a final plots and output page.

1. `generate_likelihood.py`: This evaluates the quantity $\ln(p(q, \tilde\Lambda))$ on a grid using a bounded 2d KDE from the samples for $(q, \tilde\Lambda)$. This is the log(marginalized posterior). Because it is assumed that the prior in $(q, \tilde\Lambda)$ is flat, this just becomes a pseudo-likelihood for each BNS event. Because the uncertainty in chirp mass $\mathcal{M}$ is so small, the chirp mass for each event is taken to be the mean value.
 * `--pefiles` The MCMC runs in CSV format. The column headers must contain ('mc', 'q', 'lambdat').
 * `--outfile` Name of the hdf5 file for the gridded likelihood function for each BNS. Also stores the mean chirpmass of each BNS.
 * `--qmin`, `--lambdatmax` The likelihood is approximated from the MCMC samples with a bounded_2d_kde. Its boundaries are \[qmin, 1\] and \[0, lambdatmax\]. You must make sure that no samples extend beyond these boundaries. Otherwise, the KDE method will not work correctly.
 * `--gridsize` The KDE approximation of the likelihood is gridded up. This is the number of grid points in each dimension (q, lambdat).


2. `sample_distribution.py`: This script runs the sampler `emcee` on either the prior or the posterior for the N BNS events. 
 * `--infile` The data file `pseudolikelihood.hdf5` that contains the chirp mass and gridded pseudolikelihoods for each BNS event.
 * `--outfile` Output of `emcee` in hdf5 format. Contains the chir masses, ln(posterior) and chains for each walker.
 * `--distribution` Either prior on posterior 
 * `--nwalkers` Number of walkers for `emcee` must be atleast twice the number of sampled parameters >2*(N_BNS+N_EOS). The more the merrier. 64 works well.
 * `--niter`  
 * `--nthin` Thin the final output to reduce file size of outfile.
 * `--qmin` Minimum allowed mass ratio for the prior. This doesn't have to be the same as the value used to generate the 2d_bounded_kde in `generate_likelihood.py`.
 * `--mmin` Minimum allowed individual NS mass for the prior.
 * `--mmax` Maximum allowed individual NS mass for the prior.
 * `--maxmassmin` Minimum allowed value for the maximum NS mass (M_sun) calculated for the selected EOS parameters. This should be above the maximum known mass (1.93).
 * `--maxmassmax` Maximum allowed value for the maximum NS mass (M_sun) calculated for the selected EOS parameters. This should be less than the causality constraint (3.2).
 * `--csmax` Maximum allowed speed of sound (c=1 units).
 
 
3. `calculate_ns_properties.py`: Once the mass ratios and EOS parameters are sampled for both the prior and posterior, this script calculates the following NS properties (for both the prior and posterior):
 * Maximum mass for each EOS sample.
 * Radius as a function of mass for each EOS sample.
 * $\Lambda$ as a function of mass for each EOS sample.
 * Samples for (mass1, mass2, radius1, radius2, lambda1, lambda2) for each of the N BNS events. These are derived from the sampled values of $\mathcal{M}$, $q$ and the EOS parameters. 
 
 * `--priorfile` 
 * `--posteriorfile` 
 * `--outfile` 
 * `--nburnin` Number of samples to remove from the prior and posterior files for burnin. 
 * `--nthin` Thin the data in the files. If you have already thinned the samples in `sample_distribution.py`, this will reduce the samples by another factor.
 * `--nsample` The final number of samples to use for calculating NS properties. If you set it to a really high value, it will use the maximum number of available samples available after removing burnin and thinning.


4. `generate_eos_output_page.py`: This script generates the final plots for the page `eos_output_page.html`.
 * `--infile` The ns_properties hdf5 file.
 * `--priorfile` hdf5 output file for prior emcee run.
 * `--posteriorfile` hdf5 output file for posterior emcee run.
 * `--outdir` Directory to output the final plots.

# Example 1: Reproducing the mass-radius results for GW170817

Using samples for GW170817 with flat priors in $(q, \tilde\Lambda)$, These commands should reproduce the results for GW170817 found in [arXiv:1805.11581](https://arxiv.org/abs/1805.11581). The only difference is the choice of EOS parameterization.

If you just want to see roughly what the output will look like, try these parameters (5-10 minutes):
```
NITER=100
NBURNIN=20
NTHIN=5
NSAMPLE=1000
```

If you want accurate results, try these parameters (1-2 hours):
```
NITER=1000
NBURNIN=200
NTHIN=10
NSAMPLE=5000
```

To really make sure you have properly converged chains and you are outside burnin, try 10,000 iterations (~1 day):
```
NITER=10000
NBURNIN=5000
NTHIN=10
NSAMPLE=10000
```

In [ ]:
%%bash

# MCMC runs for each BNS system
PEDATA0="RR30_reweight.csv"

# Output directory
OUTDIR="gw170817_output"

# Sampling parameters
NWALKERS=64

# Fast run
# NITER=100
# NBURNIN=20
# NTHIN=5
# NSAMPLE=1000

# Accurate run
NITER=1000
NBURNIN=200
NTHIN=10
NSAMPLE=5000

# Very accurate run
# NITER=10000
# NBURNIN=5000
# NTHIN=10
# NSAMPLE=10000

####### Start eosinference ########

# Create an output directory
mkdir $OUTDIR

# Generate pseudolikelihood function ln(p)(q, lambdat) for each BNS event
python ../bin/generate_likelihood.py \
--pefiles $PEDATA0 \
--outfile ${OUTDIR}/pseudolikelihood.hdf5 \
--qmin 0.125 --lambdatmax 10000 --gridsize 250

# Sample the prior
python ../bin/sample_distribution.py \
--infile ${OUTDIR}/pseudolikelihood.hdf5 \
--outfile ${OUTDIR}/prior.hdf5 \
--distribution prior \
--nwalkers $NWALKERS --niter $NITER --nthin 1 \
--qmin 0.125 --mmin 0.5 --mmax 3.2 \
--maxmassmin 1.93 --maxmassmax 3.2 --csmax 1.1

# Sample the posterior
python ../bin/sample_distribution.py \
--infile ${OUTDIR}/pseudolikelihood.hdf5 \
--outfile ${OUTDIR}/posterior.hdf5 \
--distribution posterior \
--nwalkers $NWALKERS --niter $NITER --nthin 1 \
--qmin 0.125 --mmin 0.5 --mmax 3.2 \
--maxmassmin 1.93 --maxmassmax 3.2 --csmax 1.1

# Calculate NS properties for the downsampled parameters (burnin removed, thinned)
python ../bin/calculate_ns_properties.py \
--priorfile ${OUTDIR}/prior.hdf5 \
--posteriorfile ${OUTDIR}/posterior.hdf5 \
--outfile ${OUTDIR}/ns_properties.hdf5 \
--nburnin $NBURNIN --nthin $NTHIN --nsample $NSAMPLE

# Generate output html page
python ../bin/generate_eos_output_page.py \
--infile ${OUTDIR}/ns_properties.hdf5 \
--priorfile ${OUTDIR}/prior.hdf5 \
--posteriorfile ${OUTDIR}/posterior.hdf5 \
--outdir ${OUTDIR}

# Example 2: Stacking parameter estimation results for 3 BNS events

If you want to stack multiple BNS observations and measuer EOS parameters from the combined likelihood, just include a parameter estimation file for each BNS event. In this example, we just use three identical copies of the GW170817 parameter estimation results.

In [ ]:
%%bash

# MCMC runs for each BNS system
PEDATA0="RR30_reweight.csv"
PEDATA1="RR30_reweight.csv"
PEDATA2="RR30_reweight.csv"

# Output directory
OUTDIR="gw170817times3_output"

# Sampling parameters
NWALKERS=64

# Fast run
# NITER=100
# NBURNIN=20
# NTHIN=5
# NSAMPLE=1000

# Accurate run
NITER=1000
NBURNIN=200
NTHIN=10
NSAMPLE=5000

# Very accurate run
# NITER=10000
# NBURNIN=5000
# NTHIN=10
# NSAMPLE=10000

####### Start eosinference ########

# Create an output directory
mkdir $OUTDIR

# Generate pseudolikelihood function ln(p)(q, lambdat) for each BNS event
python ../bin/generate_likelihood.py \
--pefiles $PEDATA0 $PEDATA1 $PEDATA2 \
--outfile ${OUTDIR}/pseudolikelihood.hdf5 \
--qmin 0.125 --lambdatmax 10000 --gridsize 250

# Sample the prior
python ../bin/sample_distribution.py \
--infile ${OUTDIR}/pseudolikelihood.hdf5 \
--outfile ${OUTDIR}/prior.hdf5 \
--distribution prior \
--nwalkers $NWALKERS --niter $NITER --nthin 1 \
--qmin 0.125 --mmin 0.5 --mmax 3.2 \
--maxmassmin 1.93 --maxmassmax 3.2 --csmax 1.1

# Sample the posterior
python ../bin/sample_distribution.py \
--infile ${OUTDIR}/pseudolikelihood.hdf5 \
--outfile ${OUTDIR}/posterior.hdf5 \
--distribution posterior \
--nwalkers $NWALKERS --niter $NITER --nthin 1 \
--qmin 0.125 --mmin 0.5 --mmax 3.2 \
--maxmassmin 1.93 --maxmassmax 3.2 --csmax 1.1

# Calculate NS properties for the downsampled parameters (burnin removed, thinned)
python ../bin/calculate_ns_properties.py \
--priorfile ${OUTDIR}/prior.hdf5 \
--posteriorfile ${OUTDIR}/posterior.hdf5 \
--outfile ${OUTDIR}/ns_properties.hdf5 \
--nburnin $NBURNIN --nthin $NTHIN --nsample $NSAMPLE

# Generate output html page
python ../bin/generate_eos_output_page.py \
--infile ${OUTDIR}/ns_properties.hdf5 \
--priorfile ${OUTDIR}/prior.hdf5 \
--posteriorfile ${OUTDIR}/posterior.hdf5 \
--outdir ${OUTDIR}

# Example 3: Using lower *AND* upper limit prior on the maximum NS mass

Many people believe that GW170817 promptly collapsed to a black hole after merger. If this is the case, then we have a decent estimate that the maximum NS mass $M_{\rm max}$ is below the gravitational (*not* baryonic) mass of the combined 2 NSs in GW170817. We can take this into account by reducing the allowed range for the NS maximum mass in the prior. For example, if we think the gravitational mass of the remnant black hole is $2.2M_\odot$ and we also know that a $1.93M_\odot$ star exists, then we can use the following bounds in the prior: `--maxmassmin 1.93 --maxmassmax 2.2`. You can also set the prior on each NS mass to `--mmax 2.2` as well, but this is not necessary.

Note: If there was no mass ejection, the baryonic mass of the remnant is the sum of the baryonic masses of the components. From this, we can calculate the gravitational mass of the remnant BH. 

In [ ]:
%%bash

# MCMC runs for each BNS system
PEDATA0="RR30_reweight.csv"
FINALBHMASS=2.2

# Output directory
OUTDIR="gw170817bhremnant_output"

# Sampling parameters
NWALKERS=64

# Fast run
# NITER=100
# NBURNIN=20
# NTHIN=5
# NSAMPLE=1000

# Accurate run
NITER=1000
NBURNIN=200
NTHIN=10
NSAMPLE=5000

# Very accurate run
# NITER=10000
# NBURNIN=5000
# NTHIN=10
# NSAMPLE=10000


####### Start eosinference ########

# Create an output directory
mkdir $OUTDIR

# Generate pseudolikelihood function ln(p)(q, lambdat) for each BNS event
python ../bin/generate_likelihood.py \
--pefiles $PEDATA0 \
--outfile ${OUTDIR}/pseudolikelihood.hdf5 \
--qmin 0.125 --lambdatmax 10000 --gridsize 250

# Sample the prior
python ../bin/sample_distribution.py \
--infile ${OUTDIR}/pseudolikelihood.hdf5 \
--outfile ${OUTDIR}/prior.hdf5 \
--distribution prior \
--nwalkers $NWALKERS --niter $NITER --nthin 1 \
--qmin 0.125 --mmin 0.5 --mmax $FINALBHMASS \
--maxmassmin 1.93 --maxmassmax $FINALBHMASS --csmax 1.1

# Sample the posterior
python ../bin/sample_distribution.py \
--infile ${OUTDIR}/pseudolikelihood.hdf5 \
--outfile ${OUTDIR}/posterior.hdf5 \
--distribution posterior \
--nwalkers $NWALKERS --niter $NITER --nthin 1 \
--qmin 0.125 --mmin 0.5 --mmax $FINALBHMASS \
--maxmassmin 1.93 --maxmassmax $FINALBHMASS --csmax 1.1

# Calculate NS properties for the downsampled parameters (burnin removed, thinned)
python ../bin/calculate_ns_properties.py \
--priorfile ${OUTDIR}/prior.hdf5 \
--posteriorfile ${OUTDIR}/posterior.hdf5 \
--outfile ${OUTDIR}/ns_properties.hdf5 \
--nburnin $NBURNIN --nthin $NTHIN --nsample $NSAMPLE

# Generate output html page
python ../bin/generate_eos_output_page.py \
--infile ${OUTDIR}/ns_properties.hdf5 \
--priorfile ${OUTDIR}/prior.hdf5 \
--posteriorfile ${OUTDIR}/posterior.hdf5 \
--outdir ${OUTDIR}